In [10]:
%profile "cloudbend"

%glue_version "3.0"
%iam_role "arn:aws:iam::898546127587:role/GlueSessions"
%idle_timeout 120

%worker_type "G.2X"
%number_of_workers 16

%extra_jars "s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar"

Previous profile: None
Setting new profile to: cloudbend
Setting Glue version to: 3.0
Current iam_role is None
iam_role has been set to arn:aws:iam::898546127587:role/GlueSessions.
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 120 minutes.
Previous worker type: G.1X
Setting new worker type to: G.2X
Previous number of workers: 5
Setting new number of workers to: 16
Extra jars to be included:
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-cli

In [12]:
%%configure
{
    "conf": "spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
}

You are already connected to a glueetl session 735002b1-1aeb-4ee5-9600-0cface75fdf2.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


The following configurations have been updated: {'conf': 'spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'}


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()

In [3]:
catalog = "iceberg"
bucket = "pyicebergbenchmark-icebergbucket89dd3fa6-1qhkzajoxgpzo"
prefix = "iceberg"

iceberg_conf = [
    (f"spark.sql.catalog.{catalog}", "org.apache.iceberg.spark.SparkCatalog"),
    (f"spark.sql.catalog.{catalog}.warehouse", f"s3://{bucket}/{prefix}/{catalog}/"),
    (f"spark.sql.catalog.{catalog}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog"),
    (f"spark.sql.catalog.{catalog}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO"),
]

for k, v in iceberg_conf:
    spark.conf.set(k, v)

In [6]:
df = (
    spark.read
        .option("mergeSchema", "true")
        .parquet(f"s3://{bucket}/raw/*/")
)

df.count()

3318228997


In [7]:
from pyspark.sql.functions import col

df = df.filter(col("pickup_datetime").isNotNull())

df.count()

1693491462


In [8]:
df.createOrReplaceTempView("nyc_taxi")

In [9]:
%%sql

CREATE DATABASE IF NOT EXISTS iceberg.benchmark;

++
||
++
++


In [16]:
%%sql

CREATE TABLE iceberg.benchmark.nyc_taxi
USING iceberg
PARTITIONED BY (months(pickup_datetime))
TBLPROPERTIES (
    'format-version' = '2',
    'write.parquet.compression-codec' = 'snappy',
    'write.metadata.metrics.default' = 'full'
)
AS
SELECT *
FROM nyc_taxi
ORDER BY pickup_datetime;

++
||
++
++


In [18]:
%%sql

ALTER TABLE iceberg.benchmark.nyc_taxi WRITE ORDERED BY pickup_datetime;

ParseException: 
no viable alternative at input 'ALTER TABLE iceberg.benchmark.nyc_taxi WRITE'(line 2, pos 39)

== SQL ==

ALTER TABLE iceberg.benchmark.nyc_taxi WRITE ORDERED BY pickup_datetime;
---------------------------------------^^^



In [19]:
%%sql

SELECT
    MIN(pickup_datetime) AS lower,
    MAX(pickup_datetime) AS upper
FROM iceberg.benchmark.nyc_taxi;

+-------------------+-------------------+
|              lower|              upper|
+-------------------+-------------------+
|2010-01-01 00:00:00|2023-04-30 23:59:59|
+-------------------+-------------------+


In [24]:
%%sql

SELECT COUNT(*) AS total_files
FROM iceberg.benchmark.nyc_taxi.files;

+-----------+
|total_files|
+-----------+
|        230|
+-----------+


In [29]:
%%sql

SELECT *
FROM iceberg.benchmark.nyc_taxi.files
LIMIT 10;

+-------+--------------------+-----------+-------+---------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+------------+-------------+
|content|           file_path|file_format|spec_id|partition|record_count|file_size_in_bytes|        column_sizes|        value_counts|   null_value_counts|    nan_value_counts|        lower_bounds|        upper_bounds|key_metadata|       split_offsets|equality_ids|sort_order_id|
+-------+--------------------+-----------+-------+---------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+------------+-------------+
|      0|s3://pyicebergben...|    PARQUET|      0|    {480}|    13046000|         539590486|{1 -> 22854, 2 ->...|{1 -> 13046000, 2...|{1 -> 13046000, 2...|{9 ->

In [30]:
%%sql

SELECT COUNT(*) AS total_trips
FROM iceberg.benchmark.nyc_taxi
WHERE pickup_datetime BETWEEN '2023-04-01' AND '2023-05-01';

+-----------+
|total_trips|
+-----------+
|   20391382|
+-----------+


In [31]:
%%sql

-- fill rates
SELECT
    SUM(CASE WHEN nyc_taxi.vendor_id IS NULL THEN 0 ELSE 1 END) / COUNT(*) AS vendor_id,
    SUM(CASE WHEN nyc_taxi.originating_base_num IS NULL THEN 0 ELSE 1 END) / COUNT(*) AS originating_base_num,
    SUM(CASE WHEN nyc_taxi.dispatching_base_num IS NULL THEN 0 ELSE 1 END) / COUNT(*) AS dispatching_base_num,
    SUM(CASE WHEN nyc_taxi.request_datetime IS NULL THEN 0 ELSE 1 END) / COUNT(*) AS request_datetime,
    SUM(CASE WHEN nyc_taxi.lpep_pickup_datetime IS NULL THEN 0 ELSE 1 END) / COUNT(*) AS lpep_pickup_datetime,
    SUM(CASE WHEN nyc_taxi.tpep_pickup_datetime IS NULL THEN 0 ELSE 1 END) / COUNT(*) AS tpep_pickup_datetime,
    SUM(CASE WHEN nyc_taxi.pickup_datetime IS NULL THEN 0 ELSE 1 END) / COUNT(*) AS pickup_datetime
FROM iceberg.benchmark.nyc_taxi;

+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------------+
|          vendor_id|originating_base_num|dispatching_base_num|   request_datetime|lpep_pickup_datetime|tpep_pickup_datetime|pickup_datetime|
+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------------+
|0.06585785432203141|  0.3604685170830817|  0.9341408831974377|0.49651408044713247|                 0.0|                 0.0|            1.0|
+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------------+


In [32]:
%%sql

-- distinct counts
SELECT
    COUNT(DISTINCT nyc_taxi.vendor_id) AS vendor_id,
    COUNT(DISTINCT nyc_taxi.originating_base_num) AS originating_base_num,
    COUNT(DISTINCT nyc_taxi.dispatching_base_num) AS dispatching_base_num,
    COUNT(DISTINCT nyc_taxi.pu_location_id) AS pu_location_id
FROM iceberg.benchmark.nyc_taxi;

+---------+--------------------+--------------------+--------------+
|vendor_id|originating_base_num|dispatching_base_num|pu_location_id|
+---------+--------------------+--------------------+--------------+
|        3|                  67|                1333|           529|
+---------+--------------------+--------------------+--------------+


In [14]:
%stop_session

Stopping session: 735002b1-1aeb-4ee5-9600-0cface75fdf2


Stopped session.
